In [2]:
from transformers import AutoModelForSequenceClassification, BertTokenizer, AutoTokenizer, BertModel
import torch


In [14]:
import os
files = []
for dirname, _, filenames in os.walk('./eedi-mining-misconceptions-in-mathematics'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))
misconceptions_filename = files[0]
train_filename = files[3]
test_filename = files[2]       

./eedi-mining-misconceptions-in-mathematics\misconception_mapping.csv
./eedi-mining-misconceptions-in-mathematics\sample_submission.csv
./eedi-mining-misconceptions-in-mathematics\test.csv
./eedi-mining-misconceptions-in-mathematics\train.csv


In [15]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

misc_con_data = pd.read_csv(misconceptions_filename)
misc_con_data.head()

,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...
1,1,Uses dividing fractions method for multiplying...
2,2,Believes there are 100 degrees in a full turn
3,3,Thinks a quadratic without a non variable term...
4,4,Believes addition of terms and powers of terms...


In [16]:
from huggingface_hub import notebook_login
notebook_login()

In [17]:
# Load MathBERT model and tokenizer
model_name ='tbs17/MathBERT'
#tokenizer = BertTokenizer.from_pretrained(model_name,output_hidden_states=True)
#model = BertModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


C:\Users\billl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tbs17/MathBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:


# Sample math expression
expression = "Solve the equation: 2x + 3 = 7"

# Tokenize the input
inputs = tokenizer(expression, return_tensors="pt")
print(inputs)

# Get predictions
with torch.no_grad():
    outputs = model(**inputs)

# Outputs are the model's predictions
print(outputs)


{'input_ids': tensor([[ 101, 9611, 1996, 8522, 1024, 1016, 2595, 1009, 1017, 1027, 1021,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
SequenceClassifierOutput(loss=None, logits=tensor([[-0.0643,  0.0703]]), hidden_states=None, attentions=None)


In [7]:
import torch.nn.functional as F

# Get the logits from the model outputs
logits = outputs.logits

# Convert logits to probabilities
probs = F.softmax(logits, dim=-1)

# Get the predicted class (the class with the highest probability)
predicted_class = torch.argmax(probs, dim=-1)

# Map the predicted class to the corresponding label (assuming you have a list of labels)
labels = misc_con_data['MisconceptionName'].tolist()

predicted_label = labels[predicted_class.item()]

print(f"The model predicts: {predicted_label}")

The model predicts: label_2
